#### Scraping Toronto neighbourhoods and postal codes from WikiPedia with BeautifulSoup4 and Pandas


Import pandas and beautifulsoup for scraping a html table into pd.DataFrame

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import geocoder

Define the url and BeautifulSoup4-process the html with the 'lxml' transformer into a bs-object

In [ ]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url)
page = bs(req.content,'lxml')

#### Step by step walkthrough
- find the table in the bs4 object
- read the found table with Pandas straight into a DataFrame with correct column headers
- Drop all rows that have a 'Borough' with value 'Not assigned'
- Join all Neighbourhoods per Postcode 
- Reset the index so the result returns towards a Integer-indexed DataFrame
- Verification method for 'M5A'-case
- Replace all Neighbourhood == 'Not assigned' with the value of the Borough column of that row
- Verify this alteration of the DataFrame
- Show the header of the resulting DataFrame

In [ ]:
table = page.find('table')
df = pd.read_html(str(table),header=0)[0]
#df = df.drop(['Not assigned'], axis=0)
df = df[df.Borough != 'Not assigned']
# df = pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join) )
# df = df.reset_index()
# print('verify that the \'M5A\'-case is correct : \n{}\n '.format(df[df.Postcode == 'M5A']))
# df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = df.Borough
# print('verify the Neighbourhood \'Not assigned\' method:\n{}\n'.format(df[df.Borough == 'Queen\'s Park']))
df.head(12)

In [ ]:
# print('Toronto summary : \n There are {} unique Postcodes \
# and \n {} Boroughs in the final DataFrame'.format(df.Postcode.unique().size,df.Borough.unique().size))

In [ ]:
df.shape

#### Querying with arcgis, since  since Google returned no values at all and Here API couldn't find 2 PostCodes.

In [ ]:
#read Here API credentials from csv, for privacy purposes and GitHub publication
#app_id, app_code = pd.read_csv('here.csv')
# 

In [ ]:
# retrieve Latitude and Longitude with HereAPI instead of the unreliable GoogleAPI
def getLatLong(row):
    #print('post :{}'.format(row[:]))
    #print('neigh :{}'.format(row[1]))
    # initialize your variable to None
    lat_lng_coords = None
    search_query = '{}, Toronto, Ontario'.format(row)
    # loop until you get the coordinates
    try:
        while(lat_lng_coords is None):
            #g = geocoder.here(search_query,app_id=app_id,app_code=app_code)
            g = geocoder.arcgis(search_query)
            lat_lng_coords = g.latlng
            #print('FIRST')
    except IndexError:
        latitude = 0.0
        longitude = 0.0
        print('BACKUP')
        return [latitude,longitude]

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(latitude, longitude)
    return [latitude, longitude]

Query all Postcodes and return them to a list  
Then parse the created list into a DataFrame with Named Columns and add those to the Source-DataFrame  
in the correct columns


In [ ]:
getLatLong('M1B')

In [ ]:
# coords_list = df['Postal Code'].apply(getLatLong).tolist()

## Parse the resultList into a column-named dataframe structure into New_df-columns

In [ ]:

df[['Latitude','Longitude']] = pd.DataFrame(coords_list, columns=['Latitude', 'Longitude'])


In [ ]:
df

In [ ]:
df.head(12)

In [ ]:
df.to_csv('toronto_part2.csv')